In [ ]:
"""
🎓 機械学習 vs ディープラーニング 教育用統合比較システム
東京家賃予測を通じた実践学習教材
"""

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
import matplotlib.font_manager as fmt
fmt.fontManager.addfont(r'/mnt/c/Windows/Fonts/meiryo.ttc')

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Meiryo']
plt.rcParams['axes.unicode_minus'] = False

class MLvsDLEducationalSystem:
    """機械学習とディープラーニングを比較学習する教育システム"""
    
    def __init__(self, data_path='tokyo_rent_data_v2.csv'):
        """初期化およびデータ読み込み"""
        self.df = pd.read_csv(data_path)
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.models = {}
        self.results = {}
        
        print("="*80)
        print("🎓 機械学習 vs ディープラーニング教育システム")
        print("="*80)
        print(f"データ読み込み完了: {len(self.df)} サンプル")
        
    def lesson_1_data_understanding(self):
        """レッスン1: データを理解する"""
        print("\n" + "="*80)
        print("📚 レッスン1: データを理解する")
        print("="*80)
        
        # 1. データ構造の把握
        print("\n1️⃣ データ構造:")
        print(f"   - サンプル数: {len(self.df)}")
        print(f"   - 特徴量数: {len(self.df.columns) - 1}")
        print(f"   - ターゲット変数: 家賃_円")
        
        # 2. 変数タイプの分析
        print("\n2️⃣ 変数タイプ:")
        numeric_features = ['部屋サイズ_m2', '駅距離_分', '築年数_年']
        categorical_features = ['区', '建物構造', '建物タイプ']
        
        print(f"   📊 数値型変数 ({len(numeric_features)}個):")
        for feat in numeric_features:
            print(f"      - {feat}: 平均={self.df[feat].mean():.2f}, 標準偏差={self.df[feat].std():.2f}")
        
        print(f"\n   📝 カテゴリ型変数 ({len(categorical_features)}個):")
        for feat in categorical_features:
            print(f"      - {feat}: {self.df[feat].nunique()} カテゴリ")
        
        # 3. ターゲット変数の分布
        print("\n3️⃣ ターゲット変数の分布:")
        print(f"   - 平均家賃: ¥{self.df['家賃_円'].mean():,.0f}")
        print(f"   - 中央値: ¥{self.df['家賃_円'].median():,.0f}")
        print(f"   - 標準偏差: ¥{self.df['家賃_円'].std():,.0f}")
        print(f"   - 最小/最大: ¥{self.df['家賃_円'].min():,.0f} ~ ¥{self.df['家賃_円'].max():,.0f}")
        
        # 可視化
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        
        # ターゲット分布
        axes[0, 0].hist(self.df['家賃_円'], bins=50, edgecolor='black', alpha=0.7)
        axes[0, 0].set_title('ターゲット分布（家賃）')
        axes[0, 0].set_xlabel('家賃（円）')
        axes[0, 0].set_ylabel('頻度')
        
        # 数値型変数
        for idx, feat in enumerate(numeric_features):
            ax = axes[0, idx+1] if idx < 2 else axes[1, idx-2]
            ax.scatter(self.df[feat], self.df['家賃_円'], alpha=0.5)
            ax.set_xlabel(feat)
            ax.set_ylabel('家賃（円）')
            ax.set_title(f'{feat} vs 家賃')
        
        # 区別平均価格
        ward_avg = self.df.groupby('区')['家賃_円'].mean().sort_values(ascending=False)[:10]
        axes[1, 1].barh(range(len(ward_avg)), ward_avg.values)
        axes[1, 1].set_yticks(range(len(ward_avg)))
        axes[1, 1].set_yticklabels(ward_avg.index)
        axes[1, 1].set_xlabel('平均家賃（円）')
        axes[1, 1].set_title('上位10区の平均家賃')
        
        # 相関関係
        corr_matrix = self.df[numeric_features + ['家賃_円']].corr()
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                    ax=axes[1, 2], square=True)
        axes[1, 2].set_title('相関行列')
        
        plt.tight_layout()
        plt.savefig('lesson1_data_understanding.png', dpi=150)
        plt.show()
        
        # 重要なインサイト
        print("\n💡 重要なインサイト:")
        print("   1. 区が価格に大きな影響 → カテゴリ変数の重要性")
        print("   2. 部屋サイズと価格の正の相関関係")
        print("   3. 駅距離、築年数と価格の負の相関関係")
        print("   4. ターゲット分布が右に偏っている → 変換を検討")
        
        return self.df
    
    def lesson_2_preprocessing_comparison(self):
        """レッスン2: 前処理方法の比較"""
        print("\n" + "="*80)
        print("📚 レッスン2: ML vs DL 前処理の違い")
        print("="*80)
        
        from sklearn.preprocessing import LabelEncoder, OneHotEncoder
        
        # ML用前処理
        print("\n🔧 機械学習の前処理:")
        print("   1. One-Hot Encoding: カテゴリ変数をバイナリベクトルに変換")
        print("   2. Feature Scaling: StandardScalerで正規化")
        print("   3. Feature Engineering: 手動で交互作用項を生成")
        
        # One-Hot Encoding
        df_ml = pd.get_dummies(self.df, columns=['区', '建物構造', '建物タイプ'])
        print(f"\n   元の特徴量数: {len(self.df.columns)}")
        print(f"   One-Hot後: {len(df_ml.columns)}")
        
        # DL用前処理
        print("\n🧠 ディープラーニングの前処理:")
        print("   1. Label Encoding: カテゴリを整数に変換")
        print("   2. Embedding Layer: カテゴリを学習可能なベクトルに変換")
        print("   3. Automatic Feature Learning: ネットワークが自動的に特徴を学習")
        
        # Label Encoding
        le = LabelEncoder()
        df_dl = self.df.copy()
        for col in ['区', '建物構造', '建物タイプ']:
            df_dl[f'{col}_encoded'] = le.fit_transform(self.df[col])
        
        print(f"\n   Label Encoding後: 元データ + {3} エンコード列")
        
        # 可視化
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # ML前処理の結果
        axes[0].bar(['元データ', 'One-Hot後'], 
                    [len(self.df.columns), len(df_ml.columns)],
                    color=['blue', 'red'])
        axes[0].set_ylabel('特徴量数')
        axes[0].set_title('ML前処理: 特徴量の爆発')
        axes[0].text(1, len(df_ml.columns), f'{len(df_ml.columns)} 特徴量', 
                     ha='center', va='bottom')
        
        # DL前処理の結果
        embedding_dims = {'区': 16, '建物構造': 4, '建物タイプ': 4}
        categories = ['区\n(23→16)', '構造\n(4→4)', 'タイプ\n(4→4)']
        original = [23, 4, 4]
        embedded = [16, 4, 4]
        
        x = np.arange(len(categories))
        width = 0.35
        
        bars1 = axes[1].bar(x - width/2, original, width, label='元の次元', color='blue')
        bars2 = axes[1].bar(x + width/2, embedded, width, label='埋め込み次元', color='green')
        
        axes[1].set_xlabel('カテゴリ')
        axes[1].set_ylabel('次元数')
        axes[1].set_title('DL前処理: 埋め込み次元')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(categories)
        axes[1].legend()
        
        plt.tight_layout()
        plt.savefig('lesson2_preprocessing.png', dpi=150)
        plt.show()
        
        print("\n💡 重要な違い:")
        print("   ML: 高次元疎ベクトル → 計算効率性 ↓")
        print("   DL: 低次元密ベクトル → 意味的関係の学習が可能")
        
        return df_ml, df_dl
    
    def lesson_3_model_architecture(self):
        """レッスン3: モデルアーキテクチャの比較"""
        print("\n" + "="*80)
        print("📚 レッスン3: モデルアーキテクチャの比較")
        print("="*80)
        
        print("\n🔧 従来の機械学習モデル:")
        ml_models = {
            '線形回帰': '線形関係を仮定、解釈容易',
            'Ridge/Lasso': 'L2/L1正則化で過学習防止',
            'ランダムフォレスト': 'アンサンブル、非線形、特徴量重要度',
            '勾配ブースティング': '順次的アンサンブル、高性能'
        }
        
        for model, desc in ml_models.items():
            print(f"   • {model}: {desc}")
        
        print("\n🧠 ディープラーニングアーキテクチャ:")
        print("   • 入力層: 7特徴量")
        print("   • 埋め込み層: カテゴリ変数のベクトル化")
        print("   • 隠れ層: 512→256→128 (非線形変換)")
        print("   • Attention機構: 動的な重み付け")
        print("   • 出力層: 1 (予測値)")
        
        # アーキテクチャの複雑さ比較
        complexity_data = {
            'モデル': ['線形回帰', 'Ridge', 'ランダムフォレスト', 'XGBoost', 'ニューラルネット'],
            'パラメータ数': [50, 50, 10000, 5000, 300000],
            '学習時間': [1, 1, 10, 20, 100],
            '解釈可能性': [10, 10, 6, 4, 2],
            '性能': [5, 6, 8, 9, 10]
        }
        
        df_complexity = pd.DataFrame(complexity_data)
        
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # パラメータ数
        axes[0, 0].bar(df_complexity['モデル'], df_complexity['パラメータ数'], 
                       color=['green', 'green', 'blue', 'blue', 'red'])
        axes[0, 0].set_ylabel('パラメータ数')
        axes[0, 0].set_title('モデルの複雑さ')
        axes[0, 0].set_yscale('log')
        
        # 学習時間
        axes[0, 1].bar(df_complexity['モデル'], df_complexity['学習時間'],
                       color=['green', 'green', 'blue', 'blue', 'red'])
        axes[0, 1].set_ylabel('相対学習時間')
        axes[0, 1].set_title('学習効率')
        
        # 解釈可能性 vs 性能
        axes[1, 0].scatter(df_complexity['解釈可能性'], 
                          df_complexity['性能'], s=200, alpha=0.6)
        for i, model in enumerate(df_complexity['モデル']):
            axes[1, 0].annotate(model, 
                               (df_complexity['解釈可能性'][i], 
                                df_complexity['性能'][i]),
                               ha='center', va='center')
        axes[1, 0].set_xlabel('解釈可能性')
        axes[1, 0].set_ylabel('性能')
        axes[1, 0].set_title('トレードオフ: 解釈可能性 vs 性能')
        axes[1, 0].grid(True, alpha=0.3)
        
        # モデル選択ガイド
        axes[1, 1].axis('off')
        guide_text = """
        📊 モデル選択ガイド:
        
        ✅ 線形/Ridge選択時:
        • 解釈が重要な場合
        • データが少ない場合
        • 高速な学習が必要な場合
        
        ✅ ランダムフォレスト/XGBoost選択時:
        • 適度な複雑さ
        • 特徴量重要度が必要
        • 非線形関係が存在
        
        ✅ ディープラーニング選択時:
        • 大量のデータ
        • 複雑なパターン
        • 最高性能が必要
        """
        axes[1, 1].text(0.1, 0.9, guide_text, transform=axes[1, 1].transAxes,
                       fontsize=11, verticalalignment='top')
        
        plt.tight_layout()
        plt.savefig('lesson3_architecture.png', dpi=150)
        plt.show()
        
        print("\n💡 重要なインサイト:")
        print("   • 複雑さ ↑ = 性能 ↑ しかし 解釈可能性 ↓")
        print("   • データサイズと問題の複雑さに応じて選択")
        print("   • ノーフリーランチ定理: すべての問題に最適なモデルは存在しない")
    
    def lesson_4_training_comparison(self):
        """レッスン4: 学習プロセスの比較"""
        print("\n" + "="*80)
        print("📚 レッスン4: 学習プロセスの比較")
        print("="*80)
        
        # データ準備
        from sklearn.linear_model import LinearRegression, Ridge
        from sklearn.ensemble import RandomForestRegressor
        
        # 簡単な前処理
        X = pd.get_dummies(self.df.drop('家賃_円', axis=1))
        y = self.df['家賃_円'].values
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # 学習曲線の比較
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        models = {
            '線形回帰': LinearRegression(),
            'Ridge回帰': Ridge(alpha=10),
            'ランダムフォレスト': RandomForestRegressor(n_estimators=50, random_state=42)
        }
        
        for idx, (name, model) in enumerate(models.items()):
            # 学習曲線
            train_sizes, train_scores, val_scores = learning_curve(
                model, X_train_scaled, y_train, cv=5,
                train_sizes=np.linspace(0.1, 1.0, 10),
                scoring='neg_mean_squared_error'
            )
            
            ax = axes[idx//2, idx%2]
            ax.plot(train_sizes, -train_scores.mean(axis=1), 
                   'o-', label='訓練', linewidth=2)
            ax.plot(train_sizes, -val_scores.mean(axis=1), 
                   'o-', label='検証', linewidth=2)
            ax.fill_between(train_sizes, 
                           -train_scores.mean(axis=1) - train_scores.std(axis=1),
                           -train_scores.mean(axis=1) + train_scores.std(axis=1),
                           alpha=0.1)
            ax.fill_between(train_sizes,
                           -val_scores.mean(axis=1) - val_scores.std(axis=1),
                           -val_scores.mean(axis=1) + val_scores.std(axis=1),
                           alpha=0.1)
            
            ax.set_xlabel('訓練サイズ')
            ax.set_ylabel('MSE')
            ax.set_title(f'{name}の学習曲線')
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        # ディープラーニングの学習曲線（シミュレーション）
        ax = axes[1, 1]
        epochs = np.arange(1, 51)
        train_loss = 50000 * np.exp(-epochs/15) + 10000
        val_loss = 50000 * np.exp(-epochs/12) + 12000
        
        ax.plot(epochs, train_loss, label='訓練損失', linewidth=2)
        ax.plot(epochs, val_loss, label='検証損失', linewidth=2)
        ax.set_xlabel('エポック')
        ax.set_ylabel('損失')
        ax.set_title('ディープラーニングの学習進行')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('lesson4_training.png', dpi=150)
        plt.show()
        
        print("\n🔧 機械学習の学習特徴:")
        print("   • 即座に収束（閉形式解）")
        print("   • ハイパーパラメータが少ない")
        print("   • クロスバリデーションで評価")
        
        print("\n🧠 ディープラーニングの学習特徴:")
        print("   • 反復的最適化（勾配降下法）")
        print("   • 多くのハイパーパラメータ")
        print("   • Early stopping、学習率スケジューリング")
        
        print("\n💡 過学習防止戦略:")
        print("   ML: 正則化（L1/L2）、木の深さ制限、アンサンブル")
        print("   DL: Dropout、Batch Norm、Weight Decay、データ拡張")
    
    def lesson_5_performance_comparison(self):
        """レッスン5: 性能比較と解釈"""
        print("\n" + "="*80)
        print("📚 レッスン5: 性能比較と解釈")
        print("="*80)
        
        # 仮想的な性能データ（実際の学習結果をシミュレーション）
        performance_data = {
            'モデル': ['線形回帰', 'Ridge', 'Lasso', 'ランダムフォレスト', 'XGBoost', 'ニューラルネット'],
            '訓練R2': [0.75, 0.74, 0.73, 0.95, 0.93, 0.96],
            'テストR2': [0.72, 0.73, 0.72, 0.85, 0.87, 0.89],
            'MAE': [15000, 14800, 15200, 11000, 10500, 9800],
            'RMSE': [20000, 19500, 20200, 15000, 14000, 13000],
            '学習時間_秒': [0.1, 0.1, 0.2, 2.5, 5.0, 30.0]
        }
        
        df_perf = pd.DataFrame(performance_data)
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        
        # R²スコア比較
        x = np.arange(len(df_perf['モデル']))
        width = 0.35
        
        bars1 = axes[0, 0].bar(x - width/2, df_perf['訓練R2'], width, 
                               label='訓練', alpha=0.8)
        bars2 = axes[0, 0].bar(x + width/2, df_perf['テストR2'], width, 
                               label='テスト', alpha=0.8)
        axes[0, 0].set_xlabel('モデル')
        axes[0, 0].set_ylabel('R²スコア')
        axes[0, 0].set_title('R²スコア比較')
        axes[0, 0].set_xticks(x)
        axes[0, 0].set_xticklabels(df_perf['モデル'], rotation=45)
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 誤差メトリクス
        axes[0, 1].bar(df_perf['モデル'], df_perf['MAE'], alpha=0.7, 
                      label='MAE', color='blue')
        axes[0, 1].set_xlabel('モデル')
        axes[0, 1].set_ylabel('誤差（円）')
        axes[0, 1].set_title('平均絶対誤差')
        axes[0, 1].set_xticklabels(df_perf['モデル'], rotation=45)
        axes[0, 1].grid(True, alpha=0.3)
        
        # 学習時間 vs 性能
        axes[0, 2].scatter(df_perf['学習時間_秒'], df_perf['テストR2'], 
                          s=200, alpha=0.6, c=range(len(df_perf)), cmap='viridis')
        for i, model in enumerate(df_perf['モデル']):
            axes[0, 2].annotate(model, 
                               (df_perf['学習時間_秒'][i], 
                                df_perf['テストR2'][i]),
                               fontsize=9)
        axes[0, 2].set_xlabel('学習時間（秒）')
        axes[0, 2].set_ylabel('テストR²')
        axes[0, 2].set_title('効率性 vs 性能')
        axes[0, 2].grid(True, alpha=0.3)
        
        # 過学習分析
        overfitting = df_perf['訓練R2'] - df_perf['テストR2']
        axes[1, 0].bar(df_perf['モデル'], overfitting, 
                      color=['green' if x < 0.05 else 'orange' if x < 0.1 else 'red' 
                             for x in overfitting])
        axes[1, 0].set_xlabel('モデル')
        axes[1, 0].set_ylabel('訓練R² - テストR²')
        axes[1, 0].set_title('過学習分析')
        axes[1, 0].axhline(y=0.1, color='r', linestyle='--', label='過学習閾値')
        axes[1, 0].set_xticklabels(df_perf['モデル'], rotation=45)
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # 予測分布（シミュレーション）
        np.random.seed(42)
        true_values = np.random.normal(100000, 30000, 100)
        
        predictions = {
            '線形回帰': true_values + np.random.normal(0, 20000, 100),
            'ランダムフォレスト': true_values + np.random.normal(0, 15000, 100),
            'ニューラルネット': true_values + np.random.normal(0, 10000, 100)
        }
        
        for model_name, pred in predictions.items():
            axes[1, 1].scatter(true_values, pred, alpha=0.5, label=model_name, s=20)
        
        axes[1, 1].plot([true_values.min(), true_values.max()], 
                       [true_values.min(), true_values.max()], 
                       'r--', label='完璧な予測')
        axes[1, 1].set_xlabel('真値（円）')
        axes[1, 1].set_ylabel('予測値（円）')
        axes[1, 1].set_title('予測精度の比較')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        # 特徴量重要度（ランダムフォレストの例）
        feature_importance = np.random.random(10) * 0.3
        feature_importance[0] = 0.4  # 区が最も重要
        feature_names = ['区', '部屋サイズ', '駅距離', '築年数', 
                        '建物構造', 'タイプ', '階数', '向き', 
                        '駐車場', 'ペット可']
        
        sorted_idx = np.argsort(feature_importance)[::-1]
        axes[1, 2].barh(range(len(sorted_idx)), feature_importance[sorted_idx])
        axes[1, 2].set_yticks(range(len(sorted_idx)))
        axes[1, 2].set_yticklabels([feature_names[i] for i in sorted_idx])
        axes[1, 2].set_xlabel('特徴量重要度')
        axes[1, 2].set_title('特徴量重要度（ランダムフォレスト）')
        axes[1, 2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('lesson5_performance.png', dpi=150)
        plt.show()
        
        print("\n📊 性能サマリー:")
        for _, row in df_perf.iterrows():
            print(f"\n{row['モデル']}:")
            print(f"   テストR²: {row['テストR2']:.3f}")
            print(f"   MAE: ¥{row['MAE']:,.0f}")
            print(f"   過学習度: {row['訓練R2'] - row['テストR2']:.3f}")
        
        print("\n💡 重要なインサイト:")
        print("   1. 単純なモデル: 安定的、解釈容易、過学習少ない")
        print("   2. アンサンブルモデル: バランスの取れた性能")
        print("   3. ディープラーニング: 最高性能、過学習リスク、学習時間長い")
        
    def lesson_6_practical_considerations(self):
        """レッスン6: 実践的な考慮事項"""
        print("\n" + "="*80)
        print("📚 レッスン6: 実践的な考慮事項")
        print("="*80)
        
        considerations = {
            '評価基準': ['データサイズ', '学習時間', '予測時間', 
                        '解釈可能性', '特徴量エンジニアリング', 'ハイパーパラメータ調整',
                        'ハードウェア要件', 'メンテナンス'],
            '従来型ML': ['少量データでも動作', '高速', '非常に高速',
                        '高い', '手動', '中程度',
                        'CPU十分', '簡単'],
            'ディープラーニング': ['大量データ必要', '低速', '高速',
                                '低い（ブラックボックス）', '自動', '複雑',
                                'GPU推奨', '複雑']
        }
        
        df_consider = pd.DataFrame(considerations)
        
        print("\n📋 意思決定チェックリスト:")
        print("\n✅ 機械学習を選択すべき時:")
        print("   □ データが1万件未満")
        print("   □ 解釈可能性が重要（規制、医療、金融）")
        print("   □ 迅速なプロトタイピングが必要")
        print("   □ 限られたコンピューティングリソース")
        print("   □ リアルタイム予測が必要")
        
        print("\n✅ ディープラーニングを選択すべき時:")
        print("   □ 大量のデータ（10万件以上）")
        print("   □ 複雑な非線形パターン")
        print("   □ 画像、テキスト、音声データ")
        print("   □ 最高性能が目標")
        print("   □ GPUリソースが利用可能")
        
        # コスト効果分析
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # データサイズ別性能
        data_sizes = [100, 500, 1000, 5000, 10000, 50000, 100000]
        ml_performance = [0.5, 0.65, 0.72, 0.78, 0.82, 0.84, 0.85]
        dl_performance = [0.3, 0.45, 0.60, 0.75, 0.83, 0.88, 0.92]
        
        axes[0].plot(data_sizes, ml_performance, 'o-', label='従来型ML', linewidth=2)
        axes[0].plot(data_sizes, dl_performance, 's-', label='ディープラーニング', linewidth=2)
        axes[0].set_xscale('log')
        axes[0].set_xlabel('訓練データサイズ')
        axes[0].set_ylabel('モデル性能（R²）')
        axes[0].set_title('性能 vs データサイズ')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        axes[0].axvline(x=10000, color='red', linestyle='--', alpha=0.5)
        axes[0].text(10000, 0.3, '交差点', rotation=90, va='bottom')
        
        # ROI分析
        categories = ['開発\n時間', '学習\nコスト', '推論\nコスト', 
                     'メンテナンス', '精度\n向上']
        ml_scores = [8, 9, 10, 8, 6]
        dl_scores = [5, 4, 7, 5, 9]
        
        angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
        ml_scores += ml_scores[:1]
        dl_scores += dl_scores[:1]
        angles += angles[:1]
        
        ax = fig.add_subplot(122, projection='polar')
        ax.plot(angles, ml_scores, 'o-', linewidth=2, label='従来型ML')
        ax.fill(angles, ml_scores, alpha=0.25)
        ax.plot(angles, dl_scores, 's-', linewidth=2, label='ディープラーニング')
        ax.fill(angles, dl_scores, alpha=0.25)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories)
        ax.set_ylim(0, 10)
        ax.set_title('コスト便益分析', pad=20)
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
        ax.grid(True)
        
        plt.tight_layout()
        plt.savefig('lesson6_practical.png', dpi=150)
        plt.show()
        
        print("\n💼 ビジネス観点:")
        print("   • MVP/POC: 機械学習で迅速に開始")
        print("   • スケールアップ: データ増加時にディープラーニング転換を検討")
        print("   • ハイブリッド: アンサンブルで両アプローチを組み合わせ")
        
        print("\n🎯 最終推奨事項:")
        print("   1. 常に簡単なモデルから開始（ベースライン）")
        print("   2. 段階的に複雑さを増加")
        print("   3. ビジネス要件を優先")
        print("   4. 技術的負債を考慮")
    
    def create_quiz(self):
        """学習確認クイズ"""
        print("\n" + "="*80)
        print("🎯 学習確認クイズ")
        print("="*80)
        
        quiz = [
            {
                'question': "Q1. カテゴリ変数が多い場合の機械学習の問題点は？",
                'options': ["A. 次元の呪い", "B. 過学習", "C. 計算複雑性の増加", "D. すべて正しい"],
                'answer': "D",
                'explanation': "One-Hot Encodingによる次元増加、疎行列、過学習リスク"
            },
            {
                'question': "Q2. ディープラーニングの埋め込み層の利点は？",
                'options': ["A. 次元削減", "B. 意味的類似性の学習", "C. 自動特徴抽出", "D. すべて正しい"],
                'answer': "D",
                'explanation': "カテゴリを低次元密ベクトルで表現、類似カテゴリは近いベクトル"
            },
            {
                'question': "Q3. データが1000件の時の最適選択は？",
                'options': ["A. ディープラーニング", "B. ランダムフォレスト", "C. 線形回帰", "D. 状況による"],
                'answer': "B",
                'explanation': "適度な複雑さ、過学習防止、良好な性能のバランス"
            },
            {
                'question': "Q4. 解釈可能性が最も重要なドメインは？",
                'options': ["A. ゲームAI", "B. 医療診断", "C. 画像フィルター", "D. 音楽推薦"],
                'answer': "B",
                'explanation': "医療、金融、法律等では意思決定の根拠が重要"
            }
        ]
        
        for q in quiz:
            print(f"\n{q['question']}")
            for opt in q['options']:
                print(f"   {opt}")
            print(f"\n   正解: {q['answer']}")
            print(f"   説明: {q['explanation']}")
        
        print("\n" + "="*80)
        print("🏆 学習完了！")
        print("="*80)
        print("\n核心メッセージ:")
        print("• 銀の弾丸はない - 完璧なモデルは存在しない")
        print("• シンプルから始める - 簡単なものから開始")
        print("• データを知る - データ理解が最優先")
        print("• トレードオフのバランス - 性能 vs 解釈可能性 vs 効率性")

In [ ]:
"""メイン教育プログラム実行"""
system = MLvsDLEducationalSystem()

# 全カリキュラム実行
print("\n🎓 機械学習 vs ディープラーニング総合教育プログラム開始\n")

# レッスン1: データ理解
input("Enterキーを押してレッスン1を開始...")
system.lesson_1_data_understanding()

# レッスン2: 前処理比較
input("\nEnterキーを押してレッスン2へ進む...")
system.lesson_2_preprocessing_comparison()

# レッスン3: モデルアーキテクチャ
input("\nEnterキーを押してレッスン3へ進む...")
system.lesson_3_model_architecture()

# レッスン4: 学習プロセス
input("\nEnterキーを押してレッスン4へ進む...")
system.lesson_4_training_comparison()

# レッスン5: 性能比較
input("\nEnterキーを押してレッスン5へ進む...")
system.lesson_5_performance_comparison()

# レッスン6: 実践的な考慮事項
input("\nEnterキーを押してレッスン6へ進む...")
system.lesson_6_practical_considerations()

# クイズ
input("\nEnterキーを押してクイズを開始...")
system.create_quiz()

print("\n" + "="*80)
print("📚 教育プログラム完了！")
print("="*80)
print("\n生成された資料:")
print("  • lesson1_data_understanding.png")
print("  • lesson2_preprocessing.png")
print("  • lesson3_architecture.png")
print("  • lesson4_training.png")
print("  • lesson5_performance.png")
print("  • lesson6_practical.png")
print("\n次のステップ:")
print("  1. 実際のデータで直接実装してみる")
print("  2. ハイパーパラメータチューニング実習")
print("  3. アンサンブル手法の探求")
print("  4. AutoMLツールの活用")